<a href="https://colab.research.google.com/github/SaraBCoutinho/Pesquisa_Analysis/blob/main/LIAR_atual_Caso3_glove_w2v_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PARTE I 

Caso 3 adota pre process 1 para CV e TF IDF e adota Pre process 2 para Glove, W2V, Fast. Dai Para o Caso 3 sao 3 situaçoes: 1-Multiclass, 2-Binarizer, 3-Sentimental Liar. 
Logo em cada uma delas vai ter X_...3_a/b/c. Além disso, vai ter X_train_3_a_proc1 e X_train_3_a_proc2. 

### Imports

In [1]:
%%capture
from numpy import load
#--Data
!pip install datasets
from datasets import load_dataset
import pandas as pd
#--Data analysis 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import STOPWORDS, WordCloud
!pip install ftfy
# import libraries
import ftfy
import nltk
import json
import re
%matplotlib inline
import pandas as pd
import csv
import numpy as np
import nltk
import seaborn as sb
import warnings

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import Binarizer

from collections import Counter
from os import listdir, makedirs
from os.path import isfile, join, splitext, split

from nltk.stem import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from nltk import pos_tag
#nltk.download('stopwords')

from wordcloud import STOPWORDS, WordCloud

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.naive_bayes import GaussianNB

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, naive_bayes, svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier # need to import xboost calssifer

warnings.filterwarnings('ignore')
np.random.seed(0)

import nltk
nltk.download('stopwords')

# stop words list set to english
stopwords_list = stopwords.words('english') # stop word list

""" Regular expression for cleaning the statements"""
import re

hashtag_re = re.compile(r"#\w+")
mention_re = re.compile(r"@\w+")
url_re = re.compile(r"(?:https?://)?(?:[-\w]+\.)+[a-zA-Z]{2,9}[-\w/#~:;.?+=&%@~]*")
extras_re = re.compile("[.;:!\'?,\"()\[\]]")
#apos_re = "\'[a-z]*"
#leftover_re = "\S+"

from unicodedata import normalize

!pip install zeugma
!pip install preprocessing
!pip install function

from zeugma.embeddings import EmbeddingTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from preprocessing import *
from function import *

#!pip install itertools
import itertools
!pip install bert_embedding
from bert_embedding import BertEmbedding

#classifiers
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier
import tensorflow as tf#--cnn
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from gensim.models import Word2Vec

from sklearn.neighbors import KNeighborsClassifier

In [2]:
!pip install transformers
import transformers
from transformers import pipeline, AutoTokenizer

     |████████████████████████████████| 3.8 MB 4.3 MB/s 
     |████████████████████████████████| 15.7 MB 40.0 MB/s 
     |████████████████████████████████| 596 kB 52.5 MB/s 
     |████████████████████████████████| 895 kB 43.5 MB/s 
     |████████████████████████████████| 6.5 MB 28.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: numpy
    Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mxnet 1.4.0 requires numpy<1.15.0,>=1.8.2, but you have numpy 1.21.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
bert-embedding 1.0.1 requires numpy==1.14.6, but yo

### Data 

In [72]:
#dataset
dataset = load_dataset('liar')

#data
train=pd.DataFrame(dataset['train'])
test=pd.DataFrame(dataset['test'])
val=pd.DataFrame(dataset['validation'])

Using custom data configuration default
Reusing dataset liar (/root/.cache/huggingface/datasets/liar/default/1.0.0/479463e757b7991eed50ffa7504d7788d6218631a484442e2098dabbf3b44514)


  0%|          | 0/3 [00:00<?, ?it/s]

Dos pre processamentos, o multiclass tem desbalanceio na classe 5 , o sentimental liar é mais desbalanceado que o pré processamento 2. 

In [4]:
#verificaçao de valores nulos - #without blank values
#example of sample (statement)
print(train['statement'][0])
print(pd.isnull(train['statement']).sum(),pd.isnull(test['statement']).sum(),pd.isnull(val['statement']).sum())
print(pd.isnull(train['label']).sum(),pd.isnull(test['label']).sum(),pd.isnull(val['label']).sum())

Says the Annies List political group supports third-trimester abortions on demand.
0 0 0
0 0 0


### Case 1 (a) : Pre process + MultiClasse 
(X_train_a,y_train_a)

This pre processing will be applied to all tecniques (CV, TFIDF, Glove, W2V, FAST) 

In [12]:
import re
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import re, SnowballStemmer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
%%time
%%time
def clean_text(text):
    translate_table = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(translate_table)
    re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\
                        .([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",
                        re.MULTILINE | re.UNICODE)
    re_ip = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
    text = re_url.sub("URL", text)
    text = re_ip.sub("IPADDRESS", text)
    text = text.lower().split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

#--tokenizer
def tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    words = [porter_stemmer.stem(word) for word in words]
    return words

# stop words list set to english
stopwords_list = stopwords.words('english') # stop word list


CPU times: user 511 µs, sys: 0 ns, total: 511 µs
Wall time: 620 µs
CPU times: user 115 ms, sys: 1.87 ms, total: 117 ms
Wall time: 177 ms


In [74]:
train['label'].unique()

array([0, 1, 2, 3, 4, 5])

In [75]:
%%time 
train['statement']=train['statement'].map(clean_text)
test['statement']=test['statement'].map(clean_text)
val['statement']=val['statement'].map(clean_text)

# specifying features and labels
X_train_a= train['statement']
X_val_a=val['statement']
X_test_a=test['statement']
y_train_a=train['label']
y_val_a=val['label']
y_test_a=test['label']

CPU times: user 6.62 s, sys: 280 ms, total: 6.9 s
Wall time: 7.03 s


In [76]:
y_train_a.unique()

array([0, 1, 2, 3, 4, 5])

### Case 1 (b) : Pre process 1 + BinariClass 
(X_train_b,y_train_b)

In [77]:
%%time 
#0 fake 1 true 
Original_labels={5:1, 4:1, 3:1, 1:0,2:0,0:0}

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [78]:
%%time 
# specifying features and labels
X_train_b= train['statement']
X_val_b=val['statement']
X_test_b=test['statement']
y_train_b=train['label'].map(Original_labels)
y_val_b=val['label'].map(Original_labels)
y_test_b=test['label'].map(Original_labels)

CPU times: user 3.65 ms, sys: 19 µs, total: 3.67 ms
Wall time: 3.05 ms


In [79]:
y_train_b.value_counts(1)

0    0.592755
1    0.407245
Name: label, dtype: float64

### Case 1 (c) : Pre process 1 + Sentimental Liar 
(X_train_c,y_train_c)

false (0), half-true(1), mostly-true(2), true (3), ,barely-true(4), pants-fire(5)

0-1-4-5:0, 2-3:1

https://huggingface.co/datasets/liar

https://arxiv.org/pdf/2009.01047v2.pdf 


In [80]:
%%time 
#0 fake 1 true 
Original_labels={5:0, 4:0, 3:1, 1:0,2:1,0:0}

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 12.2 µs


In [81]:
%%time 
# specifying features and labels
X_train_c= train['statement']
X_val_c=val['statement']
X_test_c=test['statement']
y_train_c=train['label'].map(Original_labels)
y_val_c=val['label'].map(Original_labels)
y_test_c=test['label'].map(Original_labels)

CPU times: user 4.09 ms, sys: 2 µs, total: 4.09 ms
Wall time: 4.13 ms


In [82]:
y_train_c.count()

10269

In [83]:
y_train_c.value_counts()

0    6620
1    3649
Name: label, dtype: int64

In [84]:
y_train_c.value_counts(1)

0    0.644659
1    0.355341
Name: label, dtype: float64

### Embeddings 

In [5]:
#Features
w2v = EmbeddingTransformer('word2vec')
glove = EmbeddingTransformer('glove')
fasttext = EmbeddingTransformer('fasttext')


[==================================================] 100.0% 1662.8/1662.8MB downloaded
[==================================================] 100.0% 104.8/104.8MB downloaded
[==================================================] 100.0% 958.5/958.4MB downloaded


## PARTE II

### Classificadores

In [94]:
#--Classifiers
#--SVM
def get_svm(w2v, glove, fasttext):
    return{
        'W2V': {'CLF': SVC(random_state=42, kernel='rbf', gamma=1, probability=True),'EXT': w2v,},
        'GLOVE': {'CLF': SVC(random_state=42, kernel='rbf', gamma=0.5, probability=True),'EXT':  glove,},
        'FAST': {'CLF': SVC(random_state=42, kernel='rbf', gamma=1, probability=True),'EXT': fasttext,}
    }
    
#--LR
def get_lr(glove,w2v,fasttext):
    return {
        'GLOVE': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT':  glove,},
        'W2V': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': w2v,},
        'FAST': {'CLF': LogisticRegression(random_state=42, multi_class='auto', solver='liblinear', penalty='l1'),'EXT': fasttext,}
        }
#--RF
def get_rf(w2v, glove, fasttext):
    return {
        'W2V': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50, n_jobs=-1),'EXT': fasttext,}
            }
#--NB
def get_nb(w2v, glove, fasttext):
    return {
        'W2V': {'CLF': BernoulliNB(alpha=0.5, fit_prior=True),'EXT': w2v,},
        'GLOVE': {'CLF': BernoulliNB(alpha=0.1, fit_prior=True),'EXT':  glove,},
        'FAST': {'CLF': BernoulliNB(alpha=1, fit_prior=True),'EXT': fasttext,}
    }
#--MLP
def get_mlp(w2v, glove, fasttext):
    return {
        'W2V': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT': w2v,},
        'GLOVE': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT':  glove,},
        'FAST': {'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=40, activation='relu', solver='adam'),'EXT': fasttext,}
    }
#--EXTRA
def get_extra(w2v, glove, fasttext):
    return {
        'W2V': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': ExtraTreesClassifier(random_state=42, n_estimators=50, n_jobs=-1),'EXT': fasttext,}
    }
#--KNN
def get_knn(w2v, glove, fasttext):
    return {
        'W2V': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': w2v,},
        'GLOVE': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT':  glove,},
        'FAST': {'CLF': KNeighborsClassifier(n_neighbors=5, n_jobs=-1),'EXT': fasttext,}
    }

In [95]:
def get_CNN_6(ext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, MAX_SEQUENCE_LENGTH=300, activation='sigmoid', word_embedding=False, dense=6):
    if word_embedding == False:
        X_ext = ext.get_feature_names()
        model = Word2Vec([X_ext], min_count=1, workers=1, size=300)
    else:
        model = ext.model

    word_index = tokenizer.word_index

    nb_words = min(MAX_NB_WORDS, len(word_index)) + 1

    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if word in model.wv.vocab:
            embedding_matrix[i] = model.wv.word_vec(word)

    embedding_layer = Embedding(nb_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False
                                )
    cnn = Sequential()
    cnn.add(embedding_layer)
    cnn.add(Dropout(0.2))
    cnn.add(Conv1D(64, 6, padding='valid', activation='relu', strides=1))
    cnn.add(GlobalMaxPooling1D())
    cnn.add(Dense(256))
    cnn.add(Dropout(0.2))
    cnn.add(Activation('relu'))
    cnn.add(Dense(6))
    cnn.add(Activation(activation))
    cnn.summary()
    cnn.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return cnn

def get_CNN_2(ext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, MAX_SEQUENCE_LENGTH=300, activation='sigmoid', word_embedding=False, dense=2):
    if word_embedding == False:
        X_ext = ext.get_feature_names()
        model = Word2Vec([X_ext], min_count=1, workers=1, size=300)
    else:
        model = ext.model

    word_index = tokenizer.word_index

    nb_words = min(MAX_NB_WORDS, len(word_index)) + 1

    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if word in model.wv.vocab:
            embedding_matrix[i] = model.wv.word_vec(word)

    embedding_layer = Embedding(nb_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False
                                )
    cnn = Sequential()
    cnn.add(embedding_layer)
    cnn.add(Dropout(0.2))
    cnn.add(Conv1D(64, 2, padding='valid', activation='relu', strides=1))
    cnn.add(GlobalMaxPooling1D())
    cnn.add(Dense(256))
    cnn.add(Dropout(0.2))
    cnn.add(Activation('relu'))
    cnn.add(Dense(2))
    cnn.add(Activation(activation))
    cnn.summary()
    cnn.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return cnn

In [65]:
def get_classifier(clf, X, y, ext):
    pipe_clf = Pipeline([('extractor', ext),('clf', clf)])
    pipe_clf.fit(X, y)
    return pipe_clf

In [96]:
def update_pred_proba_6(clfs, label, train, val, test):
    pred_val = pd.DataFrame()
    pred_test = pd.DataFrame()
    prob_val = pd.DataFrame()
    prob_test = pd.DataFrame()

    for ext, clf in clfs.items():
        # Predict
        df_pred_val_ = pd.DataFrame(clf.predict(val), columns=["{}-{}".format(label, ext)])
        df_pred_test_ = pd.DataFrame(clf.predict(test), columns=["{}-{}".format(label, ext)])
        
        # # Probabilidades
        cols = [
            "{}-{}-{}".format(label, ext, clf.classes_[0]), 
            "{}-{}-{}".format(label, ext, clf.classes_[1]),
            "{}-{}-{}".format(label, ext, clf.classes_[2]),
            "{}-{}-{}".format(label, ext, clf.classes_[3]),
            "{}-{}-{}".format(label, ext, clf.classes_[4]),
            "{}-{}-{}".format(label, ext, clf.classes_[5])
        ]
        df_prob_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
        df_prob_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
        pred_val = pd.concat([pred_val, df_pred_val_], axis=1, sort=False)
        pred_test = pd.concat([pred_test, df_pred_test_], axis=1, sort=False)

        prob_val = pd.concat([prob_val, df_prob_val_], axis=1, sort=False)
        prob_test = pd.concat([prob_test, df_prob_test_], axis=1, sort=False)

    return pred_val, pred_test, prob_val, prob_test

def update_pred_proba_2(clfs, label, train, val, test):
    pred_val = pd.DataFrame()
    pred_test = pd.DataFrame()
    prob_val = pd.DataFrame()
    prob_test = pd.DataFrame()

    for ext, clf in clfs.items():
        # Predict
        df_pred_val_ = pd.DataFrame(clf.predict(val), columns=["{}-{}".format(label, ext)])
        df_pred_test_ = pd.DataFrame(clf.predict(test), columns=["{}-{}".format(label, ext)])
        
        # # Probabilidades
        cols = [
            "{}-{}-{}".format(label, ext, clf.classes_[0]), 
            "{}-{}-{}".format(label, ext, clf.classes_[1])
        ]
        df_prob_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
        df_prob_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
        pred_val = pd.concat([pred_val, df_pred_val_], axis=1, sort=False)
        pred_test = pd.concat([pred_test, df_pred_test_], axis=1, sort=False)

        prob_val = pd.concat([prob_val, df_prob_val_], axis=1, sort=False)
        prob_test = pd.concat([prob_test, df_prob_test_], axis=1, sort=False)

    return pred_val, pred_test, prob_val, prob_test

#### Caso 1 (a) - 6 

In [30]:
!mkdir Data
!mkdir Data/liar

In [40]:
del train
del test
del val
#data
train=pd.DataFrame(dataset['train'])
test=pd.DataFrame(dataset['test'])
val=pd.DataFrame(dataset['validation'])

In [87]:
# load data
train.to_csv('/content/Data/liar/train_3_a.csv')
test.to_csv('/content/Data/liar/test_3_a.csv')
val.to_csv('/content/Data/liar/val_3_a.csv')
y_train=train['label']
y_train.to_csv('/content/Data/liar/train_labels_3_a.csv')
y_test=test['label']
y_test.to_csv('/content/Data/liar/test_labels_3_a.csv')
y_val=val['label']
y_val.to_csv('/content/Data/liar/val_labels_3_a.csv')

In [88]:
train_3_a=train
test_3_a=test 
val_3_a=val

In [103]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [104]:
%%time
def main():
  for db in ["liar"]:
    if db == 'liar':
      classe = 'label'
      df_pred_val = pd.DataFrame()
      df_pred_test = pd.DataFrame()
      df_prob_val = pd.DataFrame()# inicializando proba
      df_prob_test = pd.DataFrame()
      
      df_pred_val = pd.concat([df_pred_val, val_3_a['label']], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, test_3_a['label']], axis=1, sort=False)

      df_prob_val = pd.concat([df_prob_val, val_3_a['label']], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, test_3_a['label']], axis=1, sort=False)

      print('dados ok')
      
      # SVM
      classifier = get_svm(w2v, glove, fasttext)
      clfs = { }
      
      for ext, clf in classifier.items():
        clfs[ext] = get_classifier(clf['CLF'], X_train_a, y_train_a, clf['EXT'])
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_6(clfs, "SVM", X_train_a, X_val_a, X_test_a)

      print('svm parte 1 ok')
      
      # Concatenando pred
      #df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      #df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('svm parte 2 ok')

      # LR
      classifier = get_lr(glove,w2v,fasttext)
      clfs = { }
      
      for ext, clf in classifier.items():
        clfs[ext] = get_classifier(clf['CLF'],X_train_a, y_train_a, clf['EXT'])

      pred_val, pred_test, prob_val, prob_test = update_pred_proba_6(clfs, "LR", X_train_a, X_val_a, X_test_a)
      
      # Concatenando pred
      #df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      #df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('lr ok')

      # RF
      classifier = get_rf(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_a, y_train_a, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_6(clfs, "RF", X_train_a, X_val_a, X_test_a)
      # Concatenando pred
      #df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      #df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('rf ok')

      # NB
      classifier = get_nb(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_a, y_train_a, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_6(clfs, "NB", X_train_a, X_val_a, X_test_a)
      # Concatenando pred
      #df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      #df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('nb ok')

      # MLP
      classifier = get_mlp(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'],X_train_a, y_train_a, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_6(clfs, "MLP", X_train_a, X_val_a, X_test_a)
      # Concatenando pred
      #df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      #df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('mlp ok')

      # Extra
      classifier = get_extra(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'],X_train_a, y_train_a, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_6(clfs, "EXTRA", X_train_a, X_val_a, X_test_a)
      # Concatenando pred
      #df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      #df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('extra ok')

      # KNN
      classifier = get_knn(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'],X_train_a, y_train_a, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_6(clfs, "KNN", X_train_a, X_val_a, X_test_a)
      # Concatenando pred
      #df_pred_train = pd.concat([df_pred_train, pred_train], axis=1, sort=False)
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      #df_prob_train = pd.concat([df_prob_train, prob_train], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('knn ok')

      #CNN
      MAX_NB_WORDS = 20000
      MAX_SEQUENCE_LENGTH=300
      tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
      tokenizer.fit_on_texts(X_train_a) #rever por que 
      seq_train = tokenizer.texts_to_sequences(X_train_a)
      seq_val = tokenizer.texts_to_sequences(X_val_a)
      seq_test = tokenizer.texts_to_sequences(X_test_a)

      data_train = pad_sequences(seq_train,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
      data_val = pad_sequences(seq_val,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
      data_test = pad_sequences(seq_test, padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
                            
      # NET
      cnn_w2v = get_CNN_6(w2v, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=6)
      cnn_glove = get_CNN_6(glove, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True, dense=6)
      cnn_fast = get_CNN_6(fasttext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=6)
      
      y_validation=to_categorical(y_val_a)
      y_training=to_categorical(y_train_a)
      y_testing=to_categorical(y_test_a)
      
      # TRAIN
      cnn_w2v.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      cnn_glove.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      cnn_fast.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      
      #predictions and proba
      cols = ["CNN-W2V-0", "CNN-W2V-1","CNN-W2V-2", "CNN-W2V-3","CNN-W2V-4", "CNN-W2V-5"]
      df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
      df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-GLOVE-0", "CNN-GLOVE-1","CNN-GLOVE-2", "CNN-GLOVE-3","CNN-GLOVE-4", "CNN-GLOVE-5"]
      df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
      df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      
      # Probabilidades
      df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-FAST-0", "CNN-FAST-1","CNN-FAST-2", "CNN-FAST-3","CNN-FAST-4", "CNN-FAST-5"]
      df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
      df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      
      # Probabilidades
      df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False) 

      print('cnn ok')     

      # Probabilidade
      path = "./Data/{}".format(db)
      df_pred_val.to_csv("{}/pred_val_3_a.csv".format(path))
      df_pred_test.to_csv("{}/pred_test_3_a.csv".format(path))

      df_prob_val.to_csv("{}/prob_val_3_a.csv".format(path))
      df_prob_test.to_csv("{}/prob_test_3_a.csv".format(path))


CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.4 µs


In [105]:
%%time
if __name__ == "__main__":
    main()

dados ok
svm parte 1 ok
svm parte 2 ok
lr ok
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
building tree 2 of 50
building tree 1 of 50
building tree 3 of 50[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.3s

building tree 4 of 50[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s

building tree 5 of 50[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.6s

building tree 6 of 50[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.7s

building tree 7 of 50[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.9s

building tree 8 of 50[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    1.0s

building tree 9 of 50[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    1.2s

building tree 10 of 50[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.3s

building tree 11 of 50[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.5s

building tree 12 of 50[Parallel(n_jobs=-1)]: Done  10 tasks      | elaps

#### Caso 1 (b) - 2 

In [106]:
del train
del test
del val
#data
train=pd.DataFrame(dataset['train'])
test=pd.DataFrame(dataset['test'])
val=pd.DataFrame(dataset['validation'])

Original_labels={5:1, 4:1, 3:1, 1:0,2:0,0:0}


# load data
train.to_csv('/content/Data/liar/train_3_b.csv')
test.to_csv('/content/Data/liar/test_3_b.csv')
val.to_csv('/content/Data/liar/val_3_b.csv')
y_train=train['label'].map(Original_labels)
y_train.to_csv('/content/Data/liar/train_labels_3_b.csv')
y_test=test['label'].map(Original_labels)
y_test.to_csv('/content/Data/liar/test_labels_3_b.csv')
y_val=val['label'].map(Original_labels)
y_val.to_csv('/content/Data/liar/val_labels_3_b.csv')

train_3_b=train
test_3_b=test 
val_3_b=val

In [107]:
%%time
def main():
  for db in ["liar"]:
    if db == 'liar':
      classe = 'label'
      df_pred_val = pd.DataFrame()
      df_pred_test = pd.DataFrame()
      df_prob_val = pd.DataFrame()# inicializando proba
      df_prob_test = pd.DataFrame()
      
      df_pred_val = pd.concat([df_pred_val, val_3_b['label']], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, test_3_b['label']], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, val_3_b['label']], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, test_3_b['label']], axis=1, sort=False)

      print('dados ok')

      # SVM
      classifier = get_svm(w2v, glove, fasttext)
      clfs = { }
      
      for ext, clf in classifier.items():
        clfs[ext] = get_classifier(clf['CLF'], X_train_b, y_train_b, clf['EXT'])
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "SVM", X_train_b, X_val_b, X_test_b)

      print('svm parte 1 ok')
      
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('svm parte 2 ok')

      # LR
      classifier = get_lr(glove,w2v,fasttext)
      clfs = { }
      
      for ext, clf in classifier.items():
        clfs[ext] = get_classifier(clf['CLF'],X_train_b, y_train_b, clf['EXT'])

      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "LR",X_train_b, X_val_b, X_test_b)
      
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('lr ok')

      # RF
      classifier = get_rf(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_b, y_train_b, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "RF", X_train_b, X_val_b, X_test_b)
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('rf ok')

      # NB
      classifier = get_nb(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_b, y_train_b, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "NB", X_train_b, X_val_b, X_test_b)
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('nb ok')

      # MLP
      classifier = get_mlp(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_b, y_train_b, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "MLP", X_train_b, X_val_b, X_test_b)
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('mlp ok')

      # Extra
      classifier = get_extra(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_b, y_train_b, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "EXTRA", X_train_b, X_val_b, X_test_b)
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('extra ok')

      # KNN
      classifier = get_knn(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_b, y_train_b, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "KNN", X_train_b, X_val_b, X_test_b)
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('knn ok')

      #CNN
      MAX_NB_WORDS = 20000
      MAX_SEQUENCE_LENGTH=300
      tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
      tokenizer.fit_on_texts(X_train_b) #rever por que 
      seq_train = tokenizer.texts_to_sequences(X_train_b)
      seq_val = tokenizer.texts_to_sequences(X_val_b)
      seq_test = tokenizer.texts_to_sequences(X_test_b)

      data_train = pad_sequences(seq_train,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
      data_val = pad_sequences(seq_val,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
      data_test = pad_sequences(seq_test, padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
                            
      # NET
      cnn_w2v = get_CNN_2(w2v, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=2)
      cnn_glove = get_CNN_2(glove, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True, dense=2)
      cnn_fast = get_CNN_2(fasttext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=2)
      
      y_validation=to_categorical(y_val_b)
      y_training=to_categorical(y_train_b)
      y_testing=to_categorical(y_test_b)
      
      # TRAIN
      cnn_w2v.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      cnn_glove.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      cnn_fast.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      
      cols = ["CNN-W2V-0", "CNN-W2V-1"]
      df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
      df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-GLOVE-0", "CNN-GLOVE-1"]
      df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
      df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-FAST-0", "CNN-FAST-1"]
      df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
      df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)      

      print('cnn ok')     

      # Probabilidade
      path = "./Data/{}".format(db)
      df_pred_val.to_csv("{}/pred_val_3_b.csv".format(path))
      df_pred_test.to_csv("{}/pred_test_3_b.csv".format(path))

      df_prob_val.to_csv("{}/prob_val_3_b.csv".format(path))
      df_prob_test.to_csv("{}/prob_test_3_b.csv".format(path))


CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11 µs


In [108]:
%%time
if __name__ == "__main__":
    main()

dados ok
svm parte 1 ok
svm parte 2 ok
lr ok
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
building tree 1 of 50
building tree 2 of 50
building tree 3 of 50[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.3s

building tree 4 of 50[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s

building tree 5 of 50[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.6s

building tree 6 of 50[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.7s

building tree 7 of 50[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.9s

building tree 8 of 50[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    1.0s

building tree 9 of 50[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    1.2s

building tree 10 of 50[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.3s

building tree 11 of 50[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.5s

building tree 12 of 50[Parallel(n_jobs=-1)]: Done  10 tasks      | elaps

#### Caso 1 (c) - 2 - Sentimental 

In [ ]:
%%time 
del train
del test
del val
#data
train=pd.DataFrame(dataset['train'])
test=pd.DataFrame(dataset['test'])
val=pd.DataFrame(dataset['validation'])

Original_labels={5:0, 4:0, 3:1, 1:0,2:1,0:0}

# load data
train.to_csv('/content/Data/liar/train_3_c.csv')
test.to_csv('/content/Data/liar/test_3_c.csv')
val.to_csv('/content/Data/liar/val_3_c.csv')
y_train=train['label'].map(Original_labels)
y_train.to_csv('/content/Data/liar/train_labels_3_c.csv')
y_test=test['label'].map(Original_labels)
y_test.to_csv('/content/Data/liar/test_labels_3_c.csv')
y_val= val['label'].map(Original_labels)
y_val.to_csv('/content/Data/liar/val_labels_3_c.csv')

train_3_c=train
test_3_c=test 
val_3_c=val

In [ ]:
%%time
def main():
  for db in ["liar"]:
    if db == 'liar':
      classe = 'label'
      df_pred_val = pd.DataFrame()
      df_pred_test = pd.DataFrame()
      df_prob_val = pd.DataFrame()# inicializando proba
      df_prob_test = pd.DataFrame()
      
      df_pred_val = pd.concat([df_pred_val, val_3_c['label']], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, test_3_c['label']], axis=1, sort=False)
      df_prob_val = pd.concat([df_prob_val, val_3_c['label']], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, test_3_c['label']], axis=1, sort=False)

      print('dados ok')

      # SVM
      classifier = get_svm(w2v, glove, fasttext)
      clfs = { }
      
      for ext, clf in classifier.items():
        clfs[ext] = get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "SVM", X_train_c, X_val_c, X_test_c)

      print('svm parte 1 ok')
      
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('svm parte 2 ok')

      # LR
      classifier = get_lr(glove,w2v,fasttext)
      clfs = { }
      
      for ext, clf in classifier.items():
        clfs[ext] = get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])

      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "LR", X_train_c, X_val_c, X_test_c)
      
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('lr ok')

      # RF
      classifier = get_rf(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "RF", X_train_c, X_val_c, X_test_c)
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('rf ok')

      # NB
      classifier = get_nb(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "NB", X_train_c, X_val_c, X_test_c)
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('nb ok')

      # MLP
      classifier = get_mlp(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])})
          
      pred_val, pred_test,prob_val, prob_test = update_pred_proba_2(clfs, "MLP", X_train_c, X_val_c, X_test_c)
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('mlp ok')

      # Extra
      classifier = get_extra(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "EXTRA", X_train_c, X_val_c, X_test_c)
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('extra ok')

      # KNN
      classifier = get_knn(w2v, glove, fasttext)
      # clfs = { }
      for ext, clf in classifier.items():
          clfs.update({ext: get_classifier(clf['CLF'], X_train_c, y_train_c, clf['EXT'])})
          
      pred_val, pred_test, prob_val, prob_test = update_pred_proba_2(clfs, "KNN", X_train_c, X_val_c, X_test_c)
      # Concatenando pred
      df_pred_val = pd.concat([df_pred_val, pred_val], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, pred_test], axis=1, sort=False)
      # Concatenando prob
      df_prob_val = pd.concat([df_prob_val, prob_val], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, prob_test], axis=1, sort=False)

      print('knn ok')

      #CNN
      MAX_NB_WORDS = 20000
      MAX_SEQUENCE_LENGTH=300
      tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
      tokenizer.fit_on_texts(X_train_c) #rever por que 
      seq_train = tokenizer.texts_to_sequences(X_train_c)
      seq_val = tokenizer.texts_to_sequences(X_val_c)
      seq_test = tokenizer.texts_to_sequences(X_test_c)

      data_train = pad_sequences(seq_train,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
      data_val = pad_sequences(seq_val,padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
      data_test = pad_sequences(seq_test, padding='pre', truncating='pre',maxlen=MAX_SEQUENCE_LENGTH)
                            
      # NET
      cnn_w2v = get_CNN_2(w2v, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=2)
      cnn_glove = get_CNN_2(glove, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True, dense=2)
      cnn_fast = get_CNN_2(fasttext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True, dense=2)
      
      y_validation=to_categorical(y_val_c)
      y_training=to_categorical(y_train_c)
      y_testing=to_categorical(y_test_c)
      
      # TRAIN
      cnn_w2v.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      cnn_glove.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      cnn_fast.fit(data_train, y_training, validation_data=(data_val, y_validation), epochs=20, batch_size=200)
      
       #predictions and proba
      cols = ["CNN-W2V-0", "CNN-W2V-1"]
      df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
      df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-GLOVE-0", "CNN-GLOVE-1"]
      df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
      df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

      cols = ["CNN-FAST-0", "CNN-FAST-1"]
      df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
      df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])
      df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
      df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)
      # Probabilidades
      df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
      df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)
      df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
      df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)       

      print('cnn ok')     

      # Probabilidade
      path = "./Data/{}".format(db)
      df_pred_val.to_csv("{}/pred_val_3_c.csv".format(path))
      df_pred_test.to_csv("{}/pred_test_3_c.csv".format(path))

      df_prob_val.to_csv("{}/prob_val_3_c.csv".format(path))
      df_prob_test.to_csv("{}/prob_test_3_c.csv".format(path))

In [ ]:
%%time 
if __name__ == "__main__":
    main()